In [136]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
sns.set()

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [108]:
def load_joke(file):
    f = open(file,'r')
    
    data = []
    ind = 0
    for line in f:
        converted_line = ([np.float(i) for i in list(line.strip('\n').strip('').split(','))])
        data.append(converted_line)

    data = np.array(data)
    f.close()
    return data

def replace_zeros(data):
    
    return cleaned_data

def extract_non_zeros(data):
    
    return cleaned_data

In [114]:
training = load_joke('./jester/train.txt')
testing = load_joke('./jester/test.txt')

In [135]:
user = training[:,1]
np.sum(user == 0)

0

In [125]:
d_vec = [1,2,5,10,20]
d = 5

u,s,vt = scipy.sparse.linalg.svds(users,k=d)

ValueError: k must be between 1 and min(A.shape), k=5